# 🎮 Reinforcement Learning: Q-Learning to PPO

From basic RL to deep reinforcement learning.

## Learning Outcomes
- Q-learning fundamentals
- Deep Q-Networks (DQN)
- Policy gradient methods (PPO)
- Practical applications

**Level**: Advanced | **Time**: 90 min | **GPU**: Optional

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from collections import deque
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Simple Environment (GridWorld)

In [ ]:
class GridWorld:
    """Simple 4x4 GridWorld environment."""
    def __init__(self, size=4):
        self.size = size
        self.goal = (size-1, size-1)
        self.state = None
        self.reset()
    
    def reset(self):
        self.state = (0, 0)
        return self._state_to_idx()
    
    def _state_to_idx(self):
        return self.state[0] * self.size + self.state[1]
    
    def step(self, action):
        # Actions: 0=up, 1=right, 2=down, 3=left
        moves = [(-1,0), (0,1), (1,0), (0,-1)]
        new_row = max(0, min(self.size-1, self.state[0] + moves[action][0]))
        new_col = max(0, min(self.size-1, self.state[1] + moves[action][1]))
        self.state = (new_row, new_col)
        
        done = self.state == self.goal
        reward = 1.0 if done else -0.01
        
        return self._state_to_idx(), reward, done

env = GridWorld(4)
print(f"Grid: {env.size}x{env.size}, Goal: {env.goal}")

## 2. Q-Learning (Tabular)

In [ ]:
class QLearningAgent:
    def __init__(self, n_states, n_actions, lr=0.1, gamma=0.99, epsilon=1.0):
        self.q_table = np.zeros((n_states, n_actions))
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.n_actions = n_actions
    
    def get_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        return np.argmax(self.q_table[state])
    
    def update(self, state, action, reward, next_state, done):
        best_next = np.max(self.q_table[next_state]) if not done else 0
        target = reward + self.gamma * best_next
        self.q_table[state, action] += self.lr * (target - self.q_table[state, action])
        
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Train Q-learning
agent = QLearningAgent(n_states=16, n_actions=4)
rewards_history = []

print("Training Q-Learning...")
for episode in range(500):
    state = env.reset()
    total_reward = 0
    
    for _ in range(100):
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        agent.update(state, action, reward, next_state, done)
        total_reward += reward
        state = next_state
        if done:
            break
    
    rewards_history.append(total_reward)
    
    if (episode + 1) % 100 == 0:
        avg_reward = np.mean(rewards_history[-100:])
        print(f"Episode {episode+1}: Avg Reward={avg_reward:.3f}, Epsilon={agent.epsilon:.3f}")

## 3. Deep Q-Network (DQN)

In [ ]:
class DQN(nn.Module):
    def __init__(self, n_states, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, n_actions)
        )
    
    def forward(self, x):
        return self.net(x)

class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        return zip(*batch)
    
    def __len__(self):
        return len(self.buffer)

class DQNAgent:
    def __init__(self, n_states, n_actions):
        self.n_actions = n_actions
        self.policy_net = DQN(n_states, n_actions).to(device)
        self.target_net = DQN(n_states, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=1e-3)
        self.buffer = ReplayBuffer()
        self.gamma = 0.99
        self.epsilon = 1.0
    
    def get_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.n_actions - 1)
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0).to(device)
            return self.policy_net(state_t).argmax().item()
    
    def update(self, batch_size=32):
        if len(self.buffer) < batch_size:
            return
        
        states, actions, rewards, next_states, dones = self.buffer.sample(batch_size)
        
        states = torch.FloatTensor(states).to(device)
        actions = torch.LongTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.FloatTensor(dones).to(device)
        
        current_q = self.policy_net(states).gather(1, actions.unsqueeze(1))
        next_q = self.target_net(next_states).max(1)[0].detach()
        target_q = rewards + self.gamma * next_q * (1 - dones)
        
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

print(f"DQN Parameters: {sum(p.numel() for p in DQN(16, 4).parameters()):,}")

## 4. Policy Gradient (REINFORCE)

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, n_states, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states, 64),
            nn.ReLU(),
            nn.Linear(64, n_actions),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, x):
        return self.net(x)

class REINFORCEAgent:
    def __init__(self, n_states, n_actions, lr=1e-3):
        self.policy = PolicyNetwork(n_states, n_actions).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = 0.99
        self.saved_log_probs = []
        self.rewards = []
    
    def get_action(self, state):
        state_t = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs = self.policy(state_t)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        self.saved_log_probs.append(dist.log_prob(action))
        return action.item()
    
    def update(self):
        R = 0
        returns = []
        for r in reversed(self.rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        
        returns = torch.tensor(returns).to(device)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        loss = []
        for log_prob, R in zip(self.saved_log_probs, returns):
            loss.append(-log_prob * R)
        
        self.optimizer.zero_grad()
        torch.stack(loss).sum().backward()
        self.optimizer.step()
        
        self.saved_log_probs = []
        self.rewards = []

print("REINFORCE agent ready")

## 5. PPO Overview

In [ ]:
# PPO is more complex - here's the key idea
print("🎯 PPO Key Concepts:")
print("  1. Actor-Critic: Value + Policy networks")
print("  2. Clipped objective: Prevents large updates")
print("  3. Multiple epochs: Better sample efficiency")
print("  4. GAE: Generalized Advantage Estimation")
print("\n📦 Libraries:")
print("  - stable-baselines3: PPO, A2C, SAC")
print("  - cleanrl: Simple reference implementations")
print("  - rllib: Distributed RL")

## 6. Model Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Algorithm': ['Q-Learning', 'DQN', 'REINFORCE', 'A2C', 'PPO', 'SAC'],
    'Type': ['Value', 'Value', 'Policy', 'Actor-Critic', 'Actor-Critic', 'Actor-Critic'],
    'Actions': ['Discrete', 'Discrete', 'Both', 'Both', 'Both', 'Continuous'],
    'Sample Eff.': ['Low', 'Medium', 'Low', 'Medium', 'High', 'High'],
    'Best For': ['Simple', 'Atari', 'Simple PG', 'Fast', 'General', 'Robotics']
})

print("📊 RL Algorithm Comparison:")
display(comparison)

## 7. Applications

In [ ]:
applications = [
    ("🎮 Games", "AlphaGo, Atari, StarCraft"),
    ("🤖 Robotics", "Manipulation, locomotion"),
    ("💰 Trading", "Portfolio optimization"),
    ("🚗 Autonomous", "Self-driving decisions"),
    ("📱 RecSys", "Content sequencing"),
    ("🏭 Industry", "Process control"),
]

print("🚀 RL Applications:")
for name, desc in applications:
    print(f"  {name}: {desc}")

## 🎯 Key Takeaways
1. Q-learning: value-based, discrete
2. DQN: deep learning + experience replay
3. Policy gradients: directly optimize policy
4. PPO: stable, general-purpose

## 📚 Further Reading
- Sutton & Barto, "Reinforcement Learning: An Introduction"
- Mnih et al., "Human-level control through deep RL" (DQN)
- Schulman et al., "Proximal Policy Optimization" (PPO)